In [1]:
from __future__ import division
from __future__ import print_function


import numpy as np
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import collections
# from tqdm import tqdm_notebook as tqdm
from datetime import datetime

from math import ceil, floor
import cv2
import sys
import json
# from sklearn.model_selection import ShuffleSplit

In [2]:
# CT_EXCEL_FILE = '/mnt/fast-disk1/mjc/AutoRecist/Codes/ScaleNASDiceLoss/tools/AutoRECIST_Evaluation_Image_And_Contour_20220826.xlsx'


# xls = pd.ExcelFile(CT_EXCEL_FILE)

# # Now you can list all sheets in the file
# print( xls.sheet_names )
# # ['house', 'house_extra', ...]

# # to read just one sheet to dataframe:
# df_all = pd.read_excel(xls, sheet_name="Sheet2")


# df = df_all # a liver lesions case

# df = df.drop_duplicates(subset=['Image File Path'], keep='first')
# print(df)


In [3]:
def window_image(img, window_center,window_width, intercept, slope):
    
#     window_center,window_width = 50 ,100
    img = (img*slope +intercept)
    img_min = window_center - window_width//2
    img_max = window_center + window_width//2
    img[img<img_min] = img_min
    img[img>img_max] = img_max
    return img 


def get_first_of_dicom_field_as_int(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value, #window width
                    data[('0028','1052')].value, #intercept
                    data[('0028','1053')].value] #slope
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]

def _normalize(img):
    if img.max() == img.min():
        return np.zeros(img.shape)-1
    return 2 * (img - img.min())/(img.max() - img.min()) - 1

def normalize_minmax(img):
    mi, ma = img.min(), img.max()
    if mi == ma:
        return np.zeros(img.shape)-1
    return 2*(img - mi) / (ma - mi) - 1

def getName(s):
    ix1 = s.rfind('/')
    ix2 = s.rfind('.')
    return s[ix1:ix2]


def _read(path, desired_size = (512,512)):
    """Will be used in DataGenerator"""

    try:
        data = pydicom.read_file(path)
        image = data.pixel_array
        window_center , window_width, intercept, slope = get_windowing(data)
        
        image_windowed = window_image(image, window_center, window_width, intercept, slope)
        img = normalize_minmax(image_windowed)

    except:
        img = np.zeros(desired_size[:2])-1
    
    if img.shape[:2] != desired_size[:2]:
        print("image shape is not desired size. Interpolation is done.")
        img = cv2.resize(img, desired_size[:2], interpolation=cv2.INTER_LINEAR)
    
    
    return img




D_dir2header_df = {}


def get_dicom_header_df(image_dir , labels = []):
    global D_dir2header_df
    if image_dir in D_dir2header_df:
        return D_dir2header_df[image_dir]

    # image_dir = row['Image File Path']


    labels = ['ImageName','InstanceNumber',
            'BitsAllocated', 'BitsStored', 'Columns', 'HighBit', 
            'ImageOrientationPatient_0', 'ImageOrientationPatient_1', 'ImageOrientationPatient_2',
            'ImageOrientationPatient_3', 'ImageOrientationPatient_4', 'ImageOrientationPatient_5',
            'ImagePositionPatient_0', 'ImagePositionPatient_1', 'ImagePositionPatient_2',
            'Modality', 'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
            'PixelSpacing_0', 'PixelSpacing_1', 'RescaleIntercept', 'RescaleSlope', 'Rows', 'SOPInstanceUID',
            'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 
            'WindowCenter', 'WindowWidth', 
        ] if not labels else labels

    data = {l: [] for l in labels}
    
    ctList = os.listdir(image_dir)
    ctList.sort()

    for image in ctList:
        if '.dcm' not in image:
            continue
        if os.path.getsize(os.path.join(image_dir, image)) < 5*1024:
            print('%s size < 5kb skiped!'%os.path.join(image_dir, image) )
            continue
        data["ImageName"].append(image)

        ds = pydicom.dcmread(os.path.join(image_dir, image))
        for metadata in ds.dir():
            if metadata not in data and metadata not in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                continue
            if metadata != "PixelData":
                metadata_values = getattr(ds, metadata)
                if type(metadata_values) == pydicom.multival.MultiValue and metadata not in ["WindowCenter", "WindowWidth"]:
                    for i, v in enumerate(metadata_values):
                        data[f"{metadata}_{i}"].append(v)  
                else:
                    if type(metadata_values) == pydicom.multival.MultiValue and metadata in ["WindowCenter", "WindowWidth"]:
                        data[metadata].append(metadata_values[0])
                    else:
                        data[metadata].append(metadata_values)

    df_image = pd.DataFrame(data).set_index("InstanceNumber")
    D_dir2header_df[image_dir] = df_image
    return df_image


# In[2]:

def InstanceNumber2file_name(df_image, num):
    return df_image.loc[num,'ImageName']

def InstanceNumber2data_element(df_image, num, label):
    return df_image.loc[num , label]

    
def get_SliceThickness(df_image):
    flag = False
    L = df_image['ImagePositionPatient_2'].tolist()
    thick = list( np.diff(L) )
    res = float( max(set(thick), key=thick.count) )
    res = -res if res < 0 else res
    
    L.sort()
    thick2 = list( np.diff(L) )
    res2 = float( max(set(thick2), key=thick2.count) )
    if res2 ==0 and res==0:
        result = 0
        flag = True
        print('Warning intv is 0')
        print(df_image['ImagePositionPatient_2'])
    if res2 == res:
        result = res
    else:
        result = res
        flag = True
        print('Warning intv may wrong',res,res2)
        print(df_image['ImagePositionPatient_2'])
    
    return result 

def InstanceNumber2windows_min_max(df_image,num):
    try:     
        WL = InstanceNumber2data_element(df_image, num, 'WindowCenter')
        WW = InstanceNumber2data_element(df_image, num, 'WindowWidth')
    except:
        print("Warning! Window Center or Width is empty! Now use default values")
        WL , WW = 250 , 1500
        
    minHU = int( WL-WW/2 )
    maxHU = minHU + int(WW)
    return [minHU , maxHU]


class ASerial:
    P=-1
    D=-1
    S=-1
    name = ''
    def __init__(self, path_str):
        self.path = path_str
        self.getP()
        self.getD()
        self.getS()
        self.convert_path()
        
    def getP(self, target = 'DeepLesion_', L=6):
        ix = self.path.rfind(target) + len(target)
        ss = self.path[ix:ix+L]
        self.P = int(ss)
        
    def getD(self, target = '/D', L=6):
        ix = self.path.rfind(target) + len(target)
        ss = self.path[ix:ix+L]
        self.D = int(ss)
        
    def getS(self, target = '/S', L=6):
        ix = self.path.rfind(target) + len(target)
        ss = self.path[ix:ix+L]
        self.S = int(ss)
        
    def convert_path(self):
        self.name = '%06d_%02d_%02d'%(self.P, self.D, self.S)




def replacer(s, newstring, index, nofail=False):
    # raise an error if index is outside of the string
    if not nofail and index not in range(len(s)):
        raise ValueError("index outside given string")

    # if not erroring, but the index is still not in the correct range..
    if index < 0:  # add it to the beginning
        return newstring + s
    if index > len(s):  # add it to the end
        return s + newstring

    # insert the new string between "slices" of the original
    return s[:index] + newstring + s[index + 1:]

def convert_file_name(name,S='/'):
    ix = name.rfind('_')
    return replacer(name,S,ix)

def file_name2id(name):
    name.replace('.png','')
    name.replace('_','')
    return int('1' + name)
    
def get_image_size( s ):
    num = list( map( int , s.split(',')))
    return num[0] , num[1]

def get_spacing( s ):
    num = list( map( float , s.split(',')))
    return num[0] , num[1] , num[2]


def get_z_position( df ):
    s = df.loc['Normalized_lesion_location']
    num = list( map( float , s.split(',')))
    return num[2]
    
def get_slice_no( df ):
    s = df.loc['Key_slice_index']
    return int(s)

def get_windows( df ):
    s = df.loc[ 'DICOM_windows']
    num = list( map( float , s.split(',')))
    return num


def get_segmentation():
    return []

def get_bbox( df ):
    s = df.loc['Bounding_boxes']
    num = list( map( float , s.split(',')))
    num[2] = num[2]-num[0]
    num[3] = num[3]-num[1]
    return num 

def get_noise( df ):
    s = df.loc['Possibly_noisy']
    num = int(s)
    return num

def get_area( df ):
    s = df.loc['Lesion_diameters_Pixel_']
    num = list( map( float , s.split(',')))
    return num[0]*num[1]
    



newcats = [{'supercategory': 'DeepLesion', 'id': 1, 'name': 'abdomen'},
           {'supercategory': 'DeepLN', 'id': 2, 'name': 'abdomen LN'},
           {'supercategory': 'DeepLesion', 'id': 3, 'name': 'adrenal'},
           {'supercategory': 'DeepLN', 'id': 4, 'name': 'axillary LN'},
           {'supercategory': 'DeepLesion', 'id': 5, 'name': 'bone'},
           {'supercategory': 'DeepLN', 'id': 6, 'name': 'inguinal LN'},
           {'supercategory': 'DeepLesion', 'id': 7, 'name': 'kidney'},
           {'supercategory': 'DeepLesion', 'id': 8, 'name': 'liver'},
           {'supercategory': 'DeepLesion', 'id': 9, 'name': 'lung'},
           {'supercategory': 'DeepLN', 'id': 10, 'name': 'mediastinum LN'},
           {'supercategory': 'DeepLN', 'id': 11, 'name': 'neck LN'},
           {'supercategory': 'DeepLesion', 'id': 12, 'name': 'ovary'},
           {'supercategory': 'DeepLesion', 'id': 13, 'name': 'pancreas'},
           {'supercategory': 'DeepLN', 'id': 14, 'name': 'pelvic LN'},
           {'supercategory': 'DeepLesion', 'id': 15, 'name': 'pelvis'},
           {'supercategory': 'DeepLesion', 'id': 16, 'name': 'pleural'},
           {'supercategory': 'DeepLN', 'id': 17, 'name': 'retroperitoneal LN'},
           {'supercategory': 'DeepLesion', 'id': 18, 'name': 'soft tissue'},
           {'supercategory': 'DeepLesion', 'id': 19, 'name': 'spleen'},
           {'supercategory': 'DeepLesion', 'id': 20, 'name': 'stomach'},
           {'supercategory': 'DeepLesion', 'id': 21, 'name': 'thyroid'} ]

def get_21_lesion_location_cls():
    D_cls = {}
    for d in newcats:
        id_ = d['id']
        name = d['name']
        D_cls[name] = id_
    return D_cls

D_cls = get_21_lesion_location_cls()

def get_category_id( location , Dict ):
    return Dict[location]


# In[4]:

def replace_png_path(s):
    cs = s.replace('AutoRecist/Inputs' , 'AutoRecist/Pngs')
    return cs




class DeepLesion():
    """
        DL class to convert annotations to COCO Json format
    """
    def __init__(self, df,image_id_start=0,annotation_id_start=0, savename='a.json'):
        self.image_id_start = image_id_start
        self.annotation_id_start = annotation_id_start
        self.df = df 
        self.info = {"year" : 2021,
                     "version" : "2.0",
                     "description" : "Covert Weasis to Json format",
                     "contributor" : "HY,JM,BZ,LS,FSA",
                     "url" : "http:// /",
                     "date_created" : "20211129"
                    }
        self.licenses = [{"id": 1,
                          "name": "Attribution-NonCommercial",
                          "url": "http:// /"
                         }]

        self.categories = newcats
        
        self.images, self.annotations = self.__get_image_annotation_pairs__(self.df)
        json_data = {"info" : self.info,
                     "images" : self.images,
                     "licenses" : self.licenses,
                     "annotations" : self.annotations,
                     "categories" : self.categories}

        with open(savename, "w") as jsonfile:
            json.dump(json_data, jsonfile, sort_keys=True, indent=4)
            
    def change_df(self , df , savename = 'temp.json'):
        self.df = df 

        self.images, self.annotations = self.__get_image_annotation_pairs__(self.df)
        json_data = {"info" : self.info,
                     "images" : self.images,
                     "licenses" : self.licenses,
                     "annotations" : self.annotations,
                     "categories" : self.categories}

        with open(savename, "w") as jsonfile:
            json.dump(json_data, jsonfile, sort_keys=True, indent=4)
            print( 'Saved %s'%savename )
        
            
    def __get_image_annotation_pairs__(self,df):
        images = []
        annotations = []
        self.file_name_dict = {}
        for i , row in df.iterrows():
            try:
                print(i)
                df_image = get_dicom_header_df( row['Image File Path'] )
                png_folder = replace_png_path(row['Image File Path'] )
                
                for one in df_image.index.values.tolist():
#                     file_name = InstanceNumber2file_name(df_image, one)
#                     file_name = os.path.join( row['Image File Path'] , file_name)
                    file_name = os.path.join(png_folder, '%03d.png'%one)
                    file_name = file_name.replace('/mnt/fast-disk1/mjc/AutoRecist/','')

                    if file_name in self.file_name_dict:
                        oneimageid = self.file_name_dict[file_name]
                    else:
                        oneimage = {}
                        oneimage['file_name'] = file_name
                        self.image_id_start += 1
                        oneimageid = self.image_id_start
                        oneimage['id'] = oneimageid

                        oneimage['height'] , oneimage['width'] = int(InstanceNumber2data_element(df_image,one,'Rows')), int( InstanceNumber2data_element(df_image,one,'Columns') )

                        oneimage['slice_no'] = int(one)
                        oneimage['spacing'] = float( InstanceNumber2data_element(df_image,one,'PixelSpacing_0') )
                        oneimage['slice_intv'] = float( get_SliceThickness(df_image) )
                        oneimage['z_position'] = 0.5
                        oneimage['windows'] = InstanceNumber2windows_min_max(df_image,one)

                        images.append(oneimage)
                        self.file_name_dict[file_name] = oneimageid


            except Exception as e: print(e)
        
        return images, annotations
            
    


In [4]:

experiment_name = 'Lesion_Q5_scalenet_seg_test'
mask_name = 'mask_1514'
arglist = ['--cfg', '../experiments/lesion_Q5/%s.yaml'%experiment_name ,  
           '--mask_path', '../evo_files/masks/%s.npy'%mask_name,  
           'TEST.MODEL_FILE', '../output/Lesion/superscalenet_seg/Lesion_Q5_superscalenet_base/data_patch_train/best.pth',
           'DATASET.ROOT','',
           'TRAIN.USE_FLIPPED',False]

In [5]:
# This notebook is to load predicted mask for hard-disk and calculate Segmentation evaluation metics.
# predicted mask is the output of ScaleNASv2 Test and save predicted mask.ipynb
# 
# cp -v /mnt/fast-data/mjc/AutoRECIST/Codes/ScaleNAS/ScaleNASv1/tools/utils_test.py .
# 
# gt box are loaded from /cache/*gt_roidb.pkl
# gt segmentation are loaded from Hao's Raw files
# 
# predictions are all_boxes and all_segms which both are loaded from mask_1988 png files.




# # This file is for segmetation metrics evaluation in 3D
# Edited by Jingchen around 06/20/2021
# This file is after ScaleNAS test which save predition into png images.
# This file load png images as predicted contours in 2D
# load cache pkl as gold-standard contours in 2D
# The stack 2D based on dicom-header to get 3D
# Evaluate 3D metics of dice, IoU, over-segmetation, and under-segmetation
# 
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')

import os
import logging
import numpy as np
import _init_paths

from config import cfg
from config import update_config
sys.path.append('/mnt/fast-disk1/mjc/utils_codes/')
from utils_test import *
from utils_test import __get_annotation__
from utils_metrics_3d import *
import cv2
from PIL import Image

import sys
sys.path.append('/mnt/fast-disk1/mjc/utils_codes/read_weasis_raw_v0.96/')
import weasis_raw_data_api as wr

def convert_name(name):
    new = name.replace('/','_')
    return new





site_list_liver = [8]
site_list_liver_lung_LNs = [2,4,6,8,9,10,11,14,17] 
site_list_LNs = [2,4,6,10,11,14,17] 

site_list = site_list_liver
user_id = 'jm4669'

cache_path = '/mnt/fast-disk1/mjc/AutoRecist/Codes/ScaleNASDiceLoss/tools/cache/'
name = 'inference'
# name = 'lesion_train'

cache_filepath = os.path.join(cache_path, name+'_gt_roidb.pkl')
# print('Loading cached gt_roidb from %s', cache_filepath)
with open(cache_filepath, 'rb') as fp:
    cached_roidb = pickle.load(fp)
    
roidb = cached_roidb



sv_dir = mask_name
sv_path = os.path.join(sv_dir, 'test_val_results')
print( 'load AI outputs from {}'.format(sv_path) )

all_boxes = [ [ np.zeros((0,5),dtype="float32") for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]
all_segms = [ [ [] for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]

for i in range(len(roidb)):

    one = roidb[i]
    onename = one['image']
    if not os.path.exists( os.path.join( sv_path, convert_name(onename) ) ):
        print(os.path.join(sv_path, convert_name(onename) ) , 'not exists!')
        continue
    pred_im = Image.open(os.path.join( sv_path, convert_name(onename) ))
    pred = np.array(pred_im)
    for j in range(cfg.DATASET.NUM_CLASSES):
        mask = np.asarray( pred==j , dtype=np.uint8)
        if np.sum(mask > 0) <= 3 :
            continue
        segmentation, bbox, area = __get_annotation__(mask , xywh = False , bbox_score=True)
        if segmentation and bbox:
            all_segms[j][i] = segmentation
            all_boxes[j][i] = bbox

print( 'finished loading AI outputs, start to constuct D_CT' )

D_CT = {}
for i , aroidb in enumerate(roidb):
    dicom_path , png_name = os.path.split(aroidb['image'])
    slice_no , _= os.path.splitext(png_name)
    slice_no = int(slice_no)
    if slice_no != aroidb['slice_no']:
        print('following slice numbers are not consistence.')
        print(dicom_path,slice_no,aroidb['slice_no'])

    segmentations = {}
    bboxes = {}
    for j in site_list:
        segmentations[j] = all_segms[j][i]
        bboxes[j] = all_boxes[j][i]

    if dicom_path not in D_CT:
        D_CT[dicom_path] = {}
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]
    else:
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]



# In[11]:

def initialize_mask_vol( D_z_index , height , width):
    V = D_z_index.values()
    shape_z = np.max(list(V)) + 1
    mask_vol = np.zeros((shape_z , height , width ) , dtype=np.uint8 )
    return mask_vol

load AI outputs from mask_1514/test_val_results
mask_1514/test_val_results/_mnt_fast-disk1_mjc_AutoRecist_Pngs_PDS_AUTO_RECIST_METNET5126_D2014_07_25_E5869_CT_S0003_5872_001.png not exists!
mask_1514/test_val_results/_mnt_fast-disk1_mjc_AutoRecist_Pngs_PDS_AUTO_RECIST_METNET5126_D2014_07_25_E5869_CT_S0003_5872_002.png not exists!
mask_1514/test_val_results/_mnt_fast-disk1_mjc_AutoRecist_Pngs_PDS_AUTO_RECIST_METNET5126_D2014_07_25_E5869_CT_S0003_5872_003.png not exists!
mask_1514/test_val_results/_mnt_fast-disk1_mjc_AutoRecist_Pngs_PDS_AUTO_RECIST_METNET5126_D2014_07_25_E5869_CT_S0003_5872_004.png not exists!
mask_1514/test_val_results/_mnt_fast-disk1_mjc_AutoRecist_Pngs_PDS_AUTO_RECIST_METNET5126_D2014_07_25_E5869_CT_S0003_5872_005.png not exists!
mask_1514/test_val_results/_mnt_fast-disk1_mjc_AutoRecist_Pngs_PDS_AUTO_RECIST_METNET5126_D2014_07_25_E5869_CT_S0003_5872_006.png not exists!
mask_1514/test_val_results/_mnt_fast-disk1_mjc_AutoRecist_Pngs_PDS_AUTO_RECIST_METNET5126_D2014_07_2

In [6]:
whos

Variable                          Type                          Data/Info
-------------------------------------------------------------------------
ASerial                           type                          <class 'utils_metrics_3d.ASerial'>
D_CT                              dict                          n=353
D_cls                             dict                          n=21
D_dir2header_df                   dict                          n=0
DeepLesion                        type                          <class '__main__.DeepLesion'>
Dict                              dict                          n=21
FROC                              function                      <function FROC at 0x7f4d09d44ae8>
HEIGHT                            int                           512
IOU                               function                      <function IOU at 0x7f4d09d44950>
Image                             module                        <module 'PIL.Image' from <...>e-packages/PIL/Image.py'>


In [7]:
SAVE_PATH = '/mnt/fast-disk1/refine_gt/'
SAVE_FOLDER_NAME = 'ScaleNASQ5_3Slices_ToRaw_TestYenFinal360'
Metrics_vol = []
keys = list(D_CT.keys())
for k in keys:
#     if 'COU-AA-302' in k:
#         site_list = site_list_LNs
#     else:
#         site_list = site_list_liver_lung_LNs

    image_series_path = k.replace('/Pngs/' , '/Inputs/')    

    df_image = get_dicom_header_df( image_series_path )
    instanceNumber_list = df_image.index.to_list()
    D_z_index = instanceNumber2Matrix_z_index(instanceNumber_list)


    oneCT = remove_single_slice_segms(D_CT[k])
    vol_pred = get_pred_vol(oneCT , site_list , D_z_index, union_mask = False)
    



    image_series = wr.dicom_header(image_series_path)
    if len(image_series):
        height = image_series[0].Rows 
        width = image_series[0].Columns 
    else:
        print('ERROR image_series has no len' , image_series_path)
    assert(len(image_series) == vol_pred.shape[0])



    vol2 = vol_pred
    vol2[vol2>1]=1
    vol_dict = seperate_vol(vol2)
    
    for tumor_index in vol_dict:
        mask_volume = vol_dict[tumor_index]
        weasis_raw_data = wr.create(image_series, mask_volume)

        file_folder = os.path.join(SAVE_PATH , SAVE_FOLDER_NAME)
        if not os.path.exists(file_folder):
            os.makedirs(file_folder)
        file_name = wr.unique(image_series, tumor_index, user_id)
        file_name = os.path.join(file_folder,file_name)
        wr.write(weasis_raw_data, file_name)

        Metrics_vol.append( [image_series_path , file_name , user_id ])



    df_metrics = pd.DataFrame(Metrics_vol, 
                              columns = ['Image File Path','Contour File Path','Uni']) 
    df_metrics.to_csv('{}.csv'.format(SAVE_FOLDER_NAME) , index=False)
    print('finished ', k )

finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/AMGEN/20020408/BAIJC/D2004_02_27/E20040227/CT/S0002
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/AMGEN/20020408/BAIJD/D2004_02_07/E20040207/CT/S0002
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/AMGEN/20020408/BAIJG/D2004_02_02/E20040202/CT/S0013
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/AMGEN/20020408/BAIJL/D2004_02_24/E20040224/CT/S3464
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/AMGEN/20020408/BAIJM/D2004_01_31/E20040131/CT/S0003
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/AMGEN/20020408/BAIJN/D2004_02_03/E20040203/CT/S0004
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/AMGEN/20020408/BAIJO/D2004_02_19/E20040219/CT/S0005
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/AMGEN/20020408/BAIJP/D2004_02_26/E20040226/CT/S0002
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/AMGEN/20020408/BAIJQ/D2004_04_27/E20040427/CT/S0007
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/AMGEN/20020408/BAIJR/D2004_02_11/E20040211/CT/S0006
finished  /mnt/fast-

/mnt/fast-data/mjc/envs/scalenas/lib/python3.6/site-packages/pydicom/pixel_data_handlers/util.py:1094: UserWarning: A value of None for (0028,0008) 'Number of Frames' is non-conformant. It's recommended that this value be changed to 1
  warnings.warn("A value of None for (0028,0008) 'Number of Frames' is "


finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5126/D2014_07_25/E5869/CT/S0003_5872
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5127/D2014_04_09/E6205/CT/S0005_6545
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5157/D2014_11_07/E0568/CT/S0002_0514
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5181/D2014_03_01/E3057/CT/S0004_2358
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5186/D2014_03_14/E5721/CT/S0002_0978
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5187/D2014_04_03/E7041/CT/S0002_1897
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5225/D2014_09_29/E3222/CT/S0002_8263
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5238/D2012_05_03/E1769/CT/S0004_7502
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET5240/D2012_05_10/E2550/CT/S0002_1630
finished  /mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_

In [8]:
df_metrics

,Image File Path,Contour File Path,Uni
0,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/refine_gt/ScaleNASQ5_3Slices_T...,jm4669
1,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/refine_gt/ScaleNASQ5_3Slices_T...,jm4669
2,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/refine_gt/ScaleNASQ5_3Slices_T...,jm4669
3,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/refine_gt/ScaleNASQ5_3Slices_T...,jm4669
4,/mnt/fast-disk1/mjc/AutoRecist/Inputs/AMGEN/20...,/mnt/fast-disk1/refine_gt/ScaleNASQ5_3Slices_T...,jm4669
...,...,...,...
2213,/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO...,/mnt/fast-disk1/refine_gt/ScaleNASQ5_3Slices_T...,jm4669
2214,/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO...,/mnt/fast-disk1/refine_gt/ScaleNASQ5_3Slices_T...,jm4669
2215,/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO...,/mnt/fast-disk1/refine_gt/ScaleNASQ5_3Slices_T...,jm4669
2216,/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO...,/mnt/fast-disk1/refine_gt/ScaleNASQ5_3Slices_T...,jm4669


In [12]:
D_CT['/mnt/fast-disk1/mjc/AutoRecist/Pngs/AMGEN/20020408/BAIJC/D2004_02_27/E20040227/CT/S0002'][36][2][8]

[[132,
  185,
  131,
  186,
  129,
  186,
  129,
  188,
  128,
  189,
  128,
  190,
  126,
  192,
  126,
  193,
  125,
  194,
  125,
  196,
  123,
  198,
  123,
  199,
  122,
  200,
  122,
  201,
  121,
  202,
  121,
  207,
  123,
  209,
  124,
  209,
  125,
  210,
  131,
  210,
  132,
  209,
  133,
  209,
  134,
  208,
  137,
  208,
  138,
  207,
  138,
  203,
  139,
  202,
  139,
  199,
  140,
  198,
  140,
  196,
  141,
  195,
  141,
  194,
  142,
  193,
  142,
  191,
  143,
  190,
  143,
  189,
  141,
  187,
  138,
  187,
  137,
  186,
  135,
  186,
  134,
  185]]

In [13]:
whos

Variable                          Type                          Data/Info
-------------------------------------------------------------------------
ASerial                           type                          <class 'utils_metrics_3d.ASerial'>
D_CT                              dict                          n=353
D_cls                             dict                          n=21
D_dir2header_df                   dict                          n=353
D_z_index                         dict                          n=221
DeepLesion                        type                          <class '__main__.DeepLesion'>
Dict                              dict                          n=21
FROC                              function                      <function FROC at 0x7f4d09d44ae8>
HEIGHT                            int                           512
IOU                               function                      <function IOU at 0x7f4d09d44950>
Image                             module        

In [15]:
slice_number = weasis_raw_data.get_slice_number()
print(slice_number)
for slice_index in range(slice_number):
    print(slice_index)
    mask_image = weasis_raw_data.get_mask_image(slice_index)
    mask_image = mask_image.flatten()
    byte_array = bytearray(mask_image)
   

21
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [23]:
bytearray([1,2,3,4
          ])

bytearray(b'\x01\x02\x03\x04')

In [20]:
!conda list

# packages in environment at /mnt/fast-data/mjc/envs/scalenas:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       1_gnu    conda-forge
argon2-cffi               20.1.0           py36h27cfd23_1    defaults
async_generator           1.10               pyhd3eb1b0_0    defaults
attrs                     20.3.0             pyhd3eb1b0_0    defaults
backcall                  0.2.0              pyhd3eb1b0_0    defaults
blas                      1.0                         mkl    defaults
bleach                    3.3.0              pyhd3eb1b0_0    defaults
blosc                     1.19.0               hd408876_0    defaults
brotli                    1.0.9                he6710b0_2    defaults
bzip2                     1.0.8                h7b6447c_0    defaults
ca-certificates           2021.10.8            ha878542_0    conda-forge
certifi            

In [9]:
!ls *.csv

Cougar302_80pts_test_metrics_vol.csv
PDS_AMGEN_20020408_22Cat_test_liver_seg_metrics_vol_20210719.csv
PDS_AMGEN_20020408_22Cat_test_liver_seg_metrics_vol_2021116.csv
PDS_AMGEN_20020408_22Cat_test_lung_seg_metrics_vol_2021116.csv
PDS_CUIMC_22Cat_test_liver_seg_metrics_vol_20210719.csv
PDS_CUIMC_22Cat_test_liver_seg_metrics_vol_20211116.csv
PDS_CUIMC_22Cat_test_lung_seg_metrics_vol_20211116.csv
RawToWeasis.csv
RawToWeasisUnionAIRadiologist_1033atXdrive.csv
RawToWeasisUnionAIRadiologist_1033.csv
RawToWeasisUnionAIRadiologist_299atXdrive.csv
RawToWeasisUnionAIRadiologist_Amgen20020408.csv
RawToWeasisUnionAIRadiologist_AmgenatXdrive.csv
RawToWeasisUnionAIRadiologist.csv
ScaleNASQ5_3Slices_ToRaw_TestYenFinal360.csv
